In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
df= pd.read_csv('/content/drive/My Drive/SDP/crime_sdp.csv')

In [ ]:
df

,Unnamed: 0,Description,Genre
0,0,Financial TV host Lee Gates and his producer P...,Crime
1,1,"When Louis Bloom, a con man desperate for work...",Crime
2,2,A girl in a small town forms an unlikely bond ...,Non Crime
3,3,After waking up and discovering that he has un...,Crime
4,4,The savage murders of three young children spa...,Crime
...,...,...,...
295,295,A group of Boston-bred gangsters set up shop i...,Crime
296,296,"With an unmanned, half-mile-long freight train...",Non Crime
297,297,As a police psychologist works to talk down an...,Crime
298,298,When aspiring model Jesse moves to Los Angeles...,Non Crime


In [ ]:
df.drop(["Unnamed: 0"],axis=1,inplace=True)

In [ ]:
df.head()

,Description,Genre
0,Financial TV host Lee Gates and his producer P...,Crime
1,"When Louis Bloom, a con man desperate for work...",Crime
2,A girl in a small town forms an unlikely bond ...,Non Crime
3,After waking up and discovering that he has un...,Crime
4,The savage murders of three young children spa...,Crime


In [ ]:
import nltk
nltk.download()
nltk.download('punkt')

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [ ] cess_esp............ CESS-ESP Treebank
  [ ] chat80.............. Chat-80 Data Files
  [ ] city_d

True

In [ ]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [ ]:
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import PorterStemmer
#ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Description'][i])
    review = review.lower()
    review = TextBlob(review)
    review= review.correct()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

We can also add the spell correction using TEXT BLOB

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()


In [ ]:
y=pd.get_dummies(df['Genre'])

In [ ]:
y


,Crime,Non Crime
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
295,1,0
296,0,1
297,1,0
298,0,1


In [ ]:
y=y.iloc[:,1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier


from sklearn.naive_bayes import MultinomialNB
crime_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=crime_detect_model.predict(X_test)



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_pred)

In [ ]:
confusion_m

array([[20,  4],
       [18, 18]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)

In [ ]:
accuracy

0.6333333333333333

In [ ]:
from sklearn import svm

clf = svm.SVC(C=16, kernel='linear', gamma='auto')
clf.fit(X_train, y_train)
clf.predict(X_test)

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0], dtype=uint8)

In [ ]:
clf.score(X_test,y_test)

0.4

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

# print(train_x)
gnb.fit(X_train, y_train)
gnb.predict(X_test)
gnb.score(X_test,y_test)

0.6833333333333333

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear', 'rbf'), 'C':[0.1,1,8,10,16,32]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 8, 10, 16, 32],
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
from sklearn import svm

clf = svm.SVC(C=1, kernel='rbf', gamma='scale')
clf.fit(X_train, y_train)
clf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [ ]:
clf.score(X_test,y_test)

0.5666666666666667

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec= DecisionTreeClassifier(random_state=0)

clf_dec.fit(X_train, y_train)

clf_dec.predict(X_test)

clf_dec.score(X_test,y_test)

0.6166666666666667

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log= LogisticRegression(random_state=0)

clf_log.fit(X_train, y_train)

clf_log.predict(X_test)

clf_log.score(X_test,y_test)

0.6333333333333333

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(n_estimators=400, random_state=0)

clf_rf.fit(X_train, y_train)

clf_rf.predict(X_test)

clf_rf.score(X_test,y_test)

0.65

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=c5ea7bab1f2e86f24ec87e8171eb149f99ccc8d72269a95761d56438b3456a6f
  Stored in directory: /tmp/pip-ephem-wheel-cache-pq3m7wy6/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import spacy

In [ ]:
import en_core_web_md
nlp = en_core_web_md.load()

In [ ]:
train=corpus[:245]
train

In [ ]:
train_x=corpus[:245]
train_y=df.Genre[:245]
test_x=corpus[245:]
test_y=df.Genre[245:]

In [ ]:
print(train_x)
print(train_y)

print(train_x[0])
print(train_y[0])

print(test_x)
print(test_y)

['financial host lee gate producer patty put extreme situation irate investor take studio', 'louis bloom con man desperate work muscle world l crime journalism blur line observer participant become star story', 'girl small town form unlikely bond recently paralyzed man taking care', 'waking discovering undergone gender assignment surgery assassin seek find doctor responsible', 'savage murder three young child spark controversial trial three teenager accused killing kiss part tetanic ritual', 'high wire artist philippe petit recruit team people help realize dream walk immense void world trade center tower', 'woman look back past nineteen men relationship life wonder one might one true love', 'identical twin gangster donald regional gray terrorist london', 'steve job take u behind scene digital revolution paint portrait man epicenter story unfold backstage three ironic product launched ending unveiling mac', 'story michael homeless dramatized boy became american football player first rou

In [ ]:
train_X=train_x
train_Y=train_y
test_X=test_x
test_Y=test_y

In [ ]:
print(type(train_X))

<class 'list'>


In [ ]:
docs = [nlp(text) for text in train_X]
train_x_word_vectors = [x.vector for x in docs]

In [ ]:
from sklearn import svm

clf_svm_wv = svm.SVC(kernel='linear')
clf_svm_wv.fit(train_x_word_vectors, train_Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
test_docs = [nlp(text) for text in test_X]
test_x_word_vectors =  [x.vector for x in test_docs]

clf_svm_wv.predict(test_x_word_vectors)

array(['Crime', 'Crime', 'Crime', 'Crime', 'Non Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Crime', 'Non Crime', 'Crime',
       'Crime', 'Crime', 'Crime', 'Non Crime', 'Crime', 'Non Crime',
       'Crime', 'Crime', 'Crime', 'Crime', 'Non Crime', 'Non Crime',
       'Non Crime', 'Crime', 'Crime', 'Crime', 'Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Crime', 'Non Crime', 'Crime',
       'Crime', 'Non Crime', 'Crime', 'Non Crime', 'Crime', 'Crime',
       'Non Crime', 'Non Crime', 'Crime', 'Non Crime', 'Crime',
       'Non Crime', 'Crime', 'Non Crime', 'Crime', 'Non Crime',
       'Non Crime'], dtype=object)

In [ ]:
clf_svm_wv.score(test_x_word_vectors,test_Y)

0.7818181818181819

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf_dec_wv= DecisionTreeClassifier(random_state=0)

clf_dec_wv.fit(train_x_word_vectors, train_Y)

clf_dec_wv.predict(test_x_word_vectors)

clf_dec_wv.score(test_x_word_vectors,test_Y)

0.6363636363636364

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_wv = GaussianNB()

# print(train_x)
gnb_wv.fit(train_x_word_vectors, train_Y)
gnb_wv.predict(test_x_word_vectors)
gnb_wv.score(test_x_word_vectors,test_Y)

0.8181818181818182

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log_wv= LogisticRegression(random_state=0)

clf_log_wv.fit(train_x_word_vectors, train_Y)

clf_log_wv.predict(test_x_word_vectors)

clf_log_wv.score(test_x_word_vectors,test_Y)

0.8363636363636363

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf_wv = RandomForestClassifier(n_estimators=400, random_state=0)

clf_rf_wv.fit(train_x_word_vectors, train_Y)

clf_rf_wv.predict(test_x_word_vectors)

clf_rf_wv.score(test_x_word_vectors,test_Y)

0.8363636363636363

# Multilabel Classification